In [1]:
import pyoxigraph as ox
import pandas as pd
import json

In [2]:
store = ox.Store("data/wety.db")
# store = ox.Store("data/test_output/wety.db")

In [44]:
def get_item_id(term, lang, n=0):
    return [s['w'].value for s in store.query(f'SELECT ?w WHERE {{ ?w <p:term> "{term}" . ?w <p:lang> "{lang}" .}}')][n]

def print_item_info(id):
    results = store.query(
        f"""SELECT ?term ?lang ?gloss WHERE {{ 
            <{id}> <p:term> ?term .
            <{id}> <p:lang> ?lang .
            OPTIONAL {{ <{id}> <p:gloss> ?gloss }}
        }} LIMIT 1"""
    )
    for result in results:
        print(result['term'].value)
        print(result['lang'].value)
        if result['gloss']: print(result['gloss'].value)
    print()

def get_item(id):
    item = {"id": id}
    results = store.query(
        f"""SELECT ?term ?lang ?url ?pos ?gloss ?isImputed ?mode ?isReconstructed ?headProgenitor ?progenitor WHERE {{ 
            <{id}> <p:term> ?term .
            <{id}> <p:lang> ?lang .
            OPTIONAL {{ <{id}> <p:url> ?url }} .
            OPTIONAL {{ <{id}> <p:pos> ?pos }} .
            OPTIONAL {{ <{id}> <p:gloss> ?gloss }} .
            OPTIONAL {{ <{id}> <p:isImputed> ?isImputed }} .
            OPTIONAL {{ <{id}> <p:mode> ?mode }} .
            OPTIONAL {{ <{id}> <p:isReconstructed> ?isReconstructed }} .
            OPTIONAL {{ <{id}> <p:headProgenitor> ?headProgenitor }} .
            OPTIONAL {{ <{id}> <p:progenitor> ?progenitor }} .
        }}"""
    )
    preds = ["term", "lang", "url", "pos", "gloss", "isImputed", "mode", "isReconstructed", "headProgenitor", "progenitor"]
    item = {p: None if p != "progenitor" else [] for p in preds} 
    for result in results:
        for p in preds:
            if result[p] is not None:
                if p == "progenitor":
                    item[p].append(result[p].value)
                else:
                    if item[p] is None:
                        item[p] = result[p].value
    return item

# we only continue expanding items if they are in only_expand_these. if this arg is none, expand all
def get_item_expansion(id, only_expand_these=None, terminal_lang=None):
    item = get_item(id)
    if item['lang'] != terminal_lang:
        children_expansion = []
        should_expand = (lambda id: id in only_expand_these) if only_expand_these else (lambda id: True)
        for child in filter(should_expand, get_children(id)):
            child_expansion = get_item_expansion(child, only_expand_these, terminal_lang)
            children_expansion.append(child_expansion)
        item['children'] = children_expansion
    return item

# optional lang arg restricts terminal children to first terms in lang
def get_item_json(id, lang=None):
    only_expand_these = get_all_lang_descendant_ancestors(id, lang) if lang else None
    item_expansion = get_item_expansion(id, only_expand_these, lang)
    return json.dumps(item_expansion, ensure_ascii=False, indent=True)

def get_head_source(id):
    results = store.query(
        f"""SELECT ?source ?order ?head WHERE {{ 
            <{id}> <p:head> ?head .
            <{id}> <p:source> ?sourceNode .
            ?sourceNode <p:item> ?source .
            ?sourceNode <p:order> ?order .
        }}"""
    )
    # for whatever reason doing this filtering outside the query is way faster
    # than using FILTER inside the query
    for result in results:
        if result['order'].value == result['head'].value:
            return result['source'].value
    return None

def get_head_root(id):
    results = store.query(
        f"""SELECT ?headProgenitor WHERE {{ 
            <{id}> <p:headProgenitor> ?headProgenitor .
        }}"""
    )
    for result in results:
        return result['headProgenitor'].value
    return None

def get_ancestors(id):
    results = store.query(
        f"""SELECT ?item WHERE {{ 
            <{id}> (<p:source>/<p:item>)+ ?item .
        }}"""
    )
    items = set()
    for result in results:
        items.add(result['item'].value)
    return items if items else None

# get all ancestors of all items that both share the same headProgenitor and are
# in lang
def get_all_lang_descendant_ancestors(headProgenitor, lang):
    results = store.query(
        f"""SELECT ?ancestor WHERE {{ 
            ?item <p:headProgenitor> <{headProgenitor}> .
            ?item (<p:source>/<p:item>)* ?ancestor .
            ?item <p:lang> ?lang .
            FILTER ( ?lang = "{lang}" ) .
        }}"""
    )
    return {result['ancestor'].value for result in results}

def get_children(id):
    return [w['child'].value for w in store.query(
        f"""SELECT ?child WHERE {{ 
            ?sourceNode <p:item> <{id}> .
            ?sourceNode <p:order> ?order .
            ?child <p:source> ?sourceNode .
            ?child <p:head> ?order . 
        }}"""
    )]

In [45]:
item = get_item_id("mind", "English", 0)
print_item_info(item)
head_root = get_head_root(item)
print_item_info(head_root)
d = get_item_json(head_root, "English")

mind
English
The ability of rational thought.

men-
Proto-Indo-European
to think, mind



In [46]:
print(d)

{
 "term": "men-",
 "lang": "Proto-Indo-European",
 "url": "https://en.wiktionary.org/wiki/Reconstruction:Proto-Indo-European%2Fmen-",
 "pos": "root",
 "gloss": "to think, mind",
 "isImputed": null,
 "mode": null,
 "isReconstructed": "true",
 "headProgenitor": null,
 "progenitor": [],
 "children": [
  {
   "term": "reminīscor",
   "lang": "Latin",
   "url": "https://en.wiktionary.org/wiki/reminiscor%23Latin",
   "pos": "verb",
   "gloss": "I recollect or remember [+genitive]",
   "isImputed": null,
   "mode": "derived",
   "isReconstructed": null,
   "headProgenitor": "w:697821",
   "progenitor": [
    "w:697821"
   ],
   "children": [
    {
     "term": "reminīscēns",
     "lang": "Latin",
     "url": "https://en.wiktionary.org/wiki/reminiscens%23Latin",
     "pos": "verb",
     "gloss": "recollecting, remembering",
     "isImputed": null,
     "mode": "form",
     "isReconstructed": null,
     "headProgenitor": "w:697821",
     "progenitor": [
      "w:697821"
     ],
     "children"

In [ ]:
data = pd.DataFrame(
    [[w['lang'].value, w['pos'].value, w['term'].value] for w in store.query(
        f"""SELECT DISTINCT ?lang ?pos ?term WHERE {{ 
            ?item <p:glossNum> ?glossNum .
            ?item <p:glossNum> ?glossNum .
            ?item <p:lang> ?lang .
            ?item <p:pos> ?pos .
            ?item <p:term> ?term .
        }}"""
    )],
    columns=["lang", "pos", "term"]
)
data.sort_values(["lang", "pos", "term"], inplace=True)
data.to_csv("data/nonzero_glossNum.csv", index=False)
print(len(data))

In [ ]:
data = pd.DataFrame(
    [[w['lang'].value, w['pos'].value, w['term'].value] for w in store.query(
        f"""SELECT DISTINCT ?lang ?pos ?term WHERE {{ 
            ?item <p:glossNum> ?glossNum .
            ?item <p:etyNum> ?etyNum .
            ?item <p:lang> ?lang .
            ?item <p:pos> ?pos .
            ?item <p:term> ?term .
        }}"""
    )],
    columns=["lang", "pos", "term"]
)
data.sort_values(["lang", "pos", "term"], inplace=True)
data.to_csv("data/nonzero_glossNum_and_etyNum.csv", index=False)
print(len(data))

In [ ]:
d = pd.read_csv("data/descendants_langs.csv", names=["lang","n"])
d["n"].sum()

In [ ]:
fas = pd.read_csv("data/feedback_arc_set_pass_1.tsv", sep="\t")